# Question 1 
- Using Enron data-set, perform 3 analysis.

# Analysis 1
- Let's analyse the data of all the employees sent mails and check how many mail communication was gone through on yearly basis

In [1]:
# import all the packages needed for the implementation
import glob
import os
import stop_words
import re
import email.utils
from operator import itemgetter
from os.path import isfile,isdir
from email.parser import Parser
import csv

In [2]:
# getting the relative path for the folder of the data
path = os.getcwd()
rootdir = os.path.join(os.getcwd(),'maildir')
pathOfDirector = os.walk(rootdir)
directories =[direc[0] for direc in pathOfDirector]

In [3]:
# this function returns all the sent paths of all the employees folders
def sentMailsPathList(directories):
    sentPath = []
    for directory in directories:
        if directory.endswith('_sent_mail'):
            sentPath.append(directory)
    return sentPath

In [4]:
sentpaths = sentMailsPathList(directories) # calling the above function

In [5]:
# yearly wise mails count is calculated to check which year got high mails
def yearlyMails(sentpaths):
    years_email = {}
    yearList = []
    for directory in sentpaths: # iterating through the sent paths
        sentMails = glob.glob(directory+ '\\' + '*') # getting all the files
        for mail in sentMails: # iterating through the files
            if os.path.isfile(mail): 
                with open(mail,'r') as text_file: # opening each file
                    email_data = text_file.read() # reading the file
                    email_data= Parser().parsestr(email_data) # reading the file as an email
                    email_date = email.utils.parsedate(email_data['date']) # checking the date of email
                    yearList.append(email_date[0]) # appending date to the list 
                    year = email_date[0] # getting the year
                    month = email_date[1] # getting the date
                    if year not in years_email: # iterating through the years
                        years_email[year] = {} # adding an empty dictionary
                        years_email[year][month] = 1 #
                        years_email[year]['counts'] = 1
                    else:
                        years_email[year]['counts'] += 1 # adding the counter
                        if month not in years_email[year]:
                            years_email[year][month] = 1
    return(years_email) 

In [6]:
years= yearlyMails(sentpaths)
print(years)

{2000: {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 'counts': 17421}, 2001: {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 'counts': 11634}, 2002: {'counts': 22, 6: 1, 7: 1}, 1999: {12: 1, 'counts': 346}}


# Analysis 2
- From the above list we can conclude that
    - for 2000 year, months 6 to 12 has highest mails
    - for 2001 year, months 1 to 5 has highest mails
- With this analysis we proceed further by analysising who are the persons involved in those highest mails 

In [29]:
# Getting the months which has highest emails
year2000Months = [6, 7, 8, 9, 10, 11, 12]
year2001Months = [1,2,3,4,5]
for year in years.items():
    if year[0] == 2000:
        for months in year[1].items():
            if months[1] > 1000:
                if months[0] != 'counts':
                    print("!count")
                    year2000Months.append(months[0])
                    print(year2000Months)
    if year[0] == 2001:
        for months in year[1].items():
            if months[1] > 1000:
                if months[0] != 'counts':
                    year2001Months.append(months[0])
                    print(year2001Months)
    

In [30]:
print("year 2000, months with highest mails:", year2000Months)
print("year 2001, months with highest mails:", year2001Months)

year 2000, months with highest mails: [6, 7, 8, 9, 10, 11, 12]
year 2001, months with highest mails: [1, 2, 3, 4, 5]


In [31]:
def emailsSentByPersons(sentpaths,year2000Months,year2001Months):
    emailSendersReceivers ={}
    email_subject = ''
    for directory in sentpaths:
            sentMails = glob.glob(directory+ '\\' + '*')       
            for mail in sentMails:
                if os.path.isfile(mail): 
                    with open(mail,'r') as text_file:
                        email_data = text_file.read()
                        email_data= Parser().parsestr(email_data)
                        email_date = email.utils.parsedate(email_data['date'])
                        year = email_date[0]
                        month = email_date[1]
                        month_2000 = year2000Months
                        month_2001 = year2001Months
                        if (year == 2000 and month in month_2000) or (year == 2001 and month in month_2001):
                            email_subject = email_subject + email_data['subject']
                            email_to = email_data['to']
                            email_from = email_data['from']
                            if email_from not in emailSendersReceivers:
                                emailSendersReceivers[email_from] = {}
                                if email_to is not None:
                                    if',' in email_to:
                                        email_to.replace('/n','')
                                        email_to_list = email_to.split(',')
                                        email_to_1 = []
                                        for x in email_to_list:
                                            x = x.strip()
                                            email_to_1.append(x)
                                        for emails in email_to_1:
                                            if emails not in emailSendersReceivers[email_from]:
                                                emailSendersReceivers[email_from][emails] = 1
                                            else:
                                                emailSendersReceivers[email_from][emails] += 1
                                    else:
                                        emailSendersReceivers[email_from][email_to] = 1
                                emailSendersReceivers[email_from]['counts'] = 1
                            else:
                                emailSendersReceivers[email_from]['counts'] += 1
                                if email_to is not None:
                                    if',' in email_to:
                                        email_to.replace('/n','')
                                        email_to_list = email_to.split(',')
                                        email_to_1 = []
                                        for x in email_to_list:
                                            x = x.strip()
                                            email_to_1.append(x)
                                        for emails in email_to_1:
                                            if emails not in emailSendersReceivers[email_from]:
                                                emailSendersReceivers[email_from][emails] = 1
                                            else:
                                                emailSendersReceivers[email_from][emails] += 1
                                    else:
                                        if email_to not in emailSendersReceivers[email_from]:
                                            emailSendersReceivers[email_from][email_to] = 1
                                        else:
                                            emailSendersReceivers[email_from][email_to] += 1
                                        
    return emailSendersReceivers

In [32]:
numberMailsSent = emailsSentByPersons(sentpaths,year2000Months,year2001Months)


In [33]:
countOfPersonsMails = []
for persons in numberMailsSent.items():
        personsCount = []
        personsCount.append(persons[0])
        personsCount.append(persons[1]['counts'])
        countOfPersonsMails.append(personsCount)
sortedCountOfPersonMails = sorted(countOfPersonsMails, key=itemgetter(1), reverse = True)
top2 = sortedCountOfPersonMails[:2]
print("Top 2 persons who's communication are high:\n")
print(top2[0][0],":",top2[0][1], '\n')
print(top2[1][0],":",top2[1][1])

Top 2 persons who's communication are high:

kay.mann@enron.com : 4096 

vince.kaminski@enron.com : 2589


# Analysis 3
- We analysed that Mann Kay and Vince Kaaminski has communicated more 

- Finally lets analyse what they were speaking about in those mails 

In [34]:
mann = []
kaminski = []
for directory in sentpaths:
        sentMails = glob.glob(directory+ '\\' + '*')       
        for mail in sentMails:
            if os.path.isfile(mail): 
                with open(mail,'r') as text_file:
                    email_data = text_file.read()
                    email_data= Parser().parsestr(email_data)
                    email_date = email.utils.parsedate(email_data['date'])
                    year = email_date[0]
                    month = email_date[1]
                    month_2000 = year2000Months
                    month_2001 = year2001Months
                    if (year == 2000 and month in month_2000) or (year == 2001 and month in month_2001):
                        if email_data['from'] == 'kay.mann@enron.com':
                            mann.append(email_data['subject'])
                        if email_data['from'] == 'vince.kaminski@enron.com':
                            kaminski.append(email_data['subject'])
                     

In [35]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer
import warnings
import gensim
import urllib.parse
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from urllib.parse import urlparse

C:\Users\chint\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [36]:
tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = get_stop_words('en')

In [37]:
# Create p_stemmer of class PorterStemmer
#p_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

In [38]:
def dataToWriteInCsvKaminski(kaminski):
    texts = []
    listOfWords = ['forwarded', 'subject', 'mail', 'Original', 'Message', 'will']

    for i in kaminski:
        raw = i.lower()
        http = re.sub(r"http\S+", ":", raw)
        forwardMessage = re.sub(r'(?m)^\-.*\n?', "", raw)
        tagRemoval = re.sub(r'subject.*|to.*|cc.*|from.*|sent by.*|.*@*,.+\n', "", forwardMessage)
        tokens = tokenizer.tokenize(tagRemoval)
        #print(tokens)
        isalpha = [i for i in tokens if  i.isalpha()]
        #print(tokens)
        shortwords = [word for word in isalpha if len(word)>3]
        #print(shortwords)
        removeWords = [i for i in shortwords if not i in listOfWords]
        # remove stop words from tokens
        stopped_tokens = [i for i in removeWords if not i in en_stop]
        # stem tokens
        stemmed_tokens = [wordnet_lemmatizer.lemmatize(i) for i in stopped_tokens]
        # add tokens to list
        #print(stemmed_tokens)
        texts.append(stemmed_tokens)
    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts)
    #print(dictionary.token2id)
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts]
    #print(corpus)
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary)
    x = ldamodel.print_topics(num_topics=5, num_words=15)

    return x

In [39]:
def dataToWriteInCsvMann(mann):
    texts = []
    listOfWords = ['forwarded', 'subject', 'mail', 'Original', 'Message', 'will']

    for i in mann:
        raw = i.lower()
        http = re.sub(r"http\S+", ":", raw)
        forwardMessage = re.sub(r'(?m)^\-.*\n?', "", raw)
        tagRemoval = re.sub(r'subject.*|to.*|cc.*|from.*|sent by.*|.*@*,.+\n', "", forwardMessage)
        tokens = tokenizer.tokenize(tagRemoval)
        #print(tokens)
        isalpha = [i for i in tokens if  i.isalpha()]
        #print(tokens)
        shortwords = [word for word in isalpha if len(word)>3]
        #print(shortwords)
        removeWords = [i for i in shortwords if not i in listOfWords]
        # remove stop words from tokens
        stopped_tokens = [i for i in removeWords if not i in en_stop]
        # stem tokens
        stemmed_tokens = [wordnet_lemmatizer.lemmatize(i) for i in stopped_tokens]
        # add tokens to list
        #print(stemmed_tokens)
        texts.append(stemmed_tokens)
    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts)
    #print(dictionary.token2id)
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts]
    #print(corpus)
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary)
    x = ldamodel.print_topics(num_topics=5, num_words=15)
    print(x)
    return x

In [40]:
# function writing the csv file with restuarants details
def csvCreation(filename, data):
    with open(filename, 'a',encoding='utf_8', errors='ignore') as outcsv: # opening a csv file as english file
        #configure writer to write standard csv file
        writer = csv.writer(outcsv, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
        # writing the headers for the file
        writer.writerow(['TOPICS', 'WORD OCCURENCES PROBABILITY'])
        for eachRes in data:
            writer.writerow([eachRes[0],eachRes[1]])

In [41]:
# check whether the directory is already created if not this function creates it
def dir_check(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path) # creates the complete folder structure

In [42]:
# creating the csv file at specified path
def output(csvPath2,csvPath1,mann,kaminski):
    if os.path.exists(csvPath1):
        os.remove(csvPath1) # removing the file if already present
        mann = dataToWriteInCsvMann(mann) # creating the restuarants data
        csvCreation(csvPath1, mann) # creating the csv file  
    else:
        mann = dataToWriteInCsvMann(mann) # creating the restuarants data
        csvCreation(csvPath1, mann) # creating the csv file
 
    if os.path.exists(csvPath2):
        os.remove(csvPath2)
        kaminski = dataToWriteInCsvKaminski(kaminski) # creating the restuarants data
        csvCreation(csvPath2, kaminski) # creating the csv file
    else:
        kaminski = dataToWriteInCsvKaminski(kaminski) # creating the restuarants data
        csvCreation(csvPath2, kaminski) # creating the csv file
        

In [43]:
csvFolder = os.path.join(path,'Question 1\Analysis 3')
dir_check(csvFolder)
csvPath1 = os.path.join(csvFolder,'Q1Analysis3_TopicModelling_Mann.csv') # path is taken
csvPath2 = os.path.join(csvFolder,'Q1Analysis3_TopicModelling_Kaminski.csv') # path is taken

In [44]:
output(csvPath2,csvPath1,mann,kaminski) # calling the function to create csv file

[(0, '0.141*"mdea" + 0.058*"committee" + 0.058*"proposal" + 0.038*"missing" + 0.034*"split" + 0.021*"pseg" + 0.020*"message" + 0.020*"conference" + 0.019*"power" + 0.019*"call" + 0.019*"rotation" + 0.016*"agreement" + 0.009*"delete" + 0.009*"previous" + 0.009*"letter"'), (1, '0.108*"delta" + 0.069*"turbine" + 0.041*"contract" + 0.040*"order" + 0.034*"change" + 0.025*"issue" + 0.025*"favor" + 0.024*"piece" + 0.021*"document" + 0.015*"blue" + 0.014*"split" + 0.014*"call" + 0.014*"enron" + 0.014*"list" + 0.014*"number"'), (2, '0.073*"pegasus" + 0.066*"letter" + 0.047*"lunch" + 0.046*"agreement" + 0.034*"marketing" + 0.028*"option" + 0.027*"city" + 0.024*"dinner" + 0.023*"execution" + 0.016*"contract" + 0.016*"update" + 0.015*"version" + 0.015*"revised" + 0.014*"final" + 0.013*"compared"'), (3, '0.133*"agreement" + 0.107*"salmon" + 0.073*"energy" + 0.072*"turbine" + 0.060*"consent" + 0.056*"legal" + 0.030*"revised" + 0.019*"purchase" + 0.019*"letter" + 0.018*"cogen" + 0.017*"austin" + 0.01